# BÀI GIẢI ĐỀ THI CUỐI KHÓA - DATA PREPROCESSING AND DATA ANALYSIS
## Đề: DL04 - K309
## Dataset: housing-prices-dataset.csv
---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
sns.set()

---
# PHẦN 1: ĐỌC DỮ LIỆU VÀ KIỂM TRA (1 điểm)
**Yêu cầu:**
- Đọc file housing-prices-dataset.csv
- Kiểm tra thông tin cơ bản (shape, info, head)
- Kiểm tra duplicate

In [ ]:
# Đọc dữ liệu
df = pd.read_csv('housing-prices-dataset.csv')

In [ ]:
# Kiểm tra shape
df.shape

In [ ]:
# Xem 5 dòng đầu
df.head()

In [ ]:
# Kiểm tra thông tin
df.info()

In [ ]:
# Kiểm tra duplicate
df.duplicated().any()

In [ ]:
# Nếu có duplicate thì xóa
if df.duplicated().any():
    df.drop_duplicates(inplace=True)
    print(f"Đã xóa duplicate. Shape mới: {df.shape}")
else:
    print("Không có duplicate")

---
# PHẦN 2: EXPLORATORY DATA ANALYSIS (4.5 điểm)
**Chọn 11 biến để phân tích:**
- Biến định tính: LotShape, Street, HouseStyle
- Biến định lượng: LotArea, YearBuilt, 1stFlrSF, 2ndFlrSF, FullBath, BedroomAbvGr, TotRmsAbvGrd, SalePrice

In [ ]:
# Chọn các biến cần phân tích
features = ['LotShape', 'Street', 'HouseStyle', 'LotArea', 'YearBuilt', 
            '1stFlrSF', '2ndFlrSF', 'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'SalePrice']
df_analysis = df[features].copy()
df_analysis.dtypes

## 2.1. Thống kê mô tả biến định lượng

In [ ]:
df_analysis[['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 
              'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'SalePrice']].describe()

## 2.2. Phân tích biến định tính

In [ ]:
print("=== LotShape ===")
print(df_analysis['LotShape'].value_counts())
print("\n=== Street ===")
print(df_analysis['Street'].value_counts())
print("\n=== HouseStyle ===")
print(df_analysis['HouseStyle'].value_counts())

## 2.3. Phân phối và boxplot SalePrice

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
sns.histplot(data=df_analysis, x='SalePrice', kde=True)
plt.title('Phân phối SalePrice')
plt.subplot(1,2,2)
sns.boxplot(data=df_analysis, y='SalePrice')
plt.title('Boxplot SalePrice')
plt.tight_layout()
plt.show()

print('Skewness:', df_analysis['SalePrice'].skew())

## 2.4. Ma trận tương quan và heatmap

In [ ]:
# Ma trận tương quan
corr_matrix = df_analysis[['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 
                            'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd', 'SalePrice']].corr()

# Heatmap
plt.figure(figsize=(10,8))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Ma trận tương quan')
plt.show()

# Tương quan với SalePrice
print("\nTương quan với SalePrice:")
print(corr_matrix['SalePrice'].sort_values(ascending=False))

## 2.5. Scatter plots

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1,3,1)
sns.scatterplot(data=df_analysis, x='1stFlrSF', y='SalePrice')
plt.title('1stFlrSF vs SalePrice')
plt.subplot(1,3,2)
sns.scatterplot(data=df_analysis, x='TotRmsAbvGrd', y='SalePrice')
plt.title('TotRmsAbvGrd vs SalePrice')
plt.subplot(1,3,3)
sns.scatterplot(data=df_analysis, x='YearBuilt', y='SalePrice')
plt.title('YearBuilt vs SalePrice')
plt.tight_layout()
plt.show()

## 2.6. Kiểm định thống kê

In [ ]:
# Chi-square test (2 biến định tính)
from scipy.stats import chi2_contingency

cross_tab = pd.crosstab(df_analysis['LotShape'], df_analysis['HouseStyle'])
chi2, p, dof, expected = chi2_contingency(cross_tab)

print("Chi-square test: LotShape vs HouseStyle")
print(f"Chi2={chi2:.4f}, p-value={p:.4f}")
print("=> Kết luận:", "PHỤ THUỘC" if p < 0.05 else "ĐỘC LẬP")

In [ ]:
# ANOVA test (biến liên tục vs phân loại)
from scipy.stats import f_oneway

groups = [group['SalePrice'] for name, group in df_analysis.groupby('HouseStyle')]
f_stat, p_value = f_oneway(*groups)

print("\nANOVA test: SalePrice vs HouseStyle")
print(f"F-statistic={f_stat:.4f}, p-value={p_value:.4f}")
print("=> Kết luận:", "Có sự khác biệt" if p_value < 0.05 else "Không khác biệt")

In [ ]:
# Barplot minh họa
plt.figure(figsize=(12,4))
plt.subplot(1,3,1)
sns.barplot(data=df_analysis, x='LotShape', y='SalePrice', ci=None)
plt.title('SalePrice theo LotShape')
plt.subplot(1,3,2)
sns.barplot(data=df_analysis, x='Street', y='SalePrice', ci=None)
plt.title('SalePrice theo Street')
plt.subplot(1,3,3)
sns.barplot(data=df_analysis, x='HouseStyle', y='SalePrice', ci=None)
plt.xticks(rotation=45)
plt.title('SalePrice theo HouseStyle')
plt.tight_layout()
plt.show()

## 2.7. Xử lý Outliers (IQR)

In [ ]:
# Tính Quantiles
Q1 = df_analysis['SalePrice'].quantile(0.25)
Q2 = df_analysis['SalePrice'].quantile(0.50)
Q3 = df_analysis['SalePrice'].quantile(0.75)
IQR = Q3 - Q1

print(f'Q1 (25%): {Q1:,.0f}')
print(f'Q2 (50%): {Q2:,.0f}')
print(f'Q3 (75%): {Q3:,.0f}')
print(f'IQR: {IQR:,.0f}')

# Upper whisker
w = 1.5
UW = Q3 + w * IQR
print(f'Upper Whisker: {UW:,.0f}')

In [ ]:
# Tìm và xóa outliers
outliers = df_analysis[df_analysis['SalePrice'] > UW]
print(f"Số outliers: {len(outliers)} ({len(outliers)/len(df_analysis)*100:.2f}%)")

df_analysis.drop(index=outliers.index, inplace=True)
print(f"Shape sau xử lý: {df_analysis.shape}")

In [ ]:
# Kiểm tra lại
sns.boxplot(data=df_analysis, y='SalePrice')
plt.title('Boxplot sau xử lý outliers')
plt.show()

---
# PHẦN 3: FEATURE ENGINEERING (2 điểm)
**Yêu cầu:**
- Chọn biến có |correlation| >= 0.3 và p-value <= 0.05
- OneHotEncoder cho biến định tính
- StandardScaler cho biến định lượng

## 3.1. Chọn biến theo correlation và p-value

In [ ]:
from scipy.stats import pearsonr

quant_vars = ['LotArea', 'YearBuilt', '1stFlrSF', '2ndFlrSF', 
              'FullBath', 'BedroomAbvGr', 'TotRmsAbvGrd']
selected_features = []

for var in quant_vars:
    corr, pval = pearsonr(df_analysis[var], df_analysis['SalePrice'])
    print(f"{var:15s}: correlation={corr:6.3f}, p-value={pval:.4f}", end='')
    
    if abs(corr) >= 0.3 and pval <= 0.05:
        selected_features.append(var)
        print(" => CHỌN")
    else:
        print(" => Loại")

print(f"\nCác biến được chọn: {selected_features}")

## 3.2. Encoding biến định tính (OneHotEncoder)

In [ ]:
# Chuẩn bị X và y
qual_vars = ['LotShape', 'Street', 'HouseStyle']
X = df_analysis[qual_vars + selected_features].copy()
y = df_analysis['SalePrice'].copy()

In [ ]:
# OneHotEncoder bằng pandas get_dummies
X_encoded = pd.get_dummies(X, columns=qual_vars, drop_first=True)
print("Shape sau encoding:", X_encoded.shape)
X_encoded.head()

## 3.3. Scaling biến định lượng (StandardScaler)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_encoded[selected_features] = scaler.fit_transform(X_encoded[selected_features])

In [ ]:
# Kiểm tra mean và std (phải xấp xỉ 0 và 1)
print("Mean:")
print(X_encoded[selected_features].mean())
print("\nStd:")
print(X_encoded[selected_features].std())

In [ ]:
# Kết quả cuối cùng
print("Shape cuối cùng:", X_encoded.shape)
X_encoded.head()

---
# PHẦN 4: LINEAR REGRESSION MODEL (1.5 điểm)
**Yêu cầu:**
- Train/test split (test_size=0.2)
- Xây dựng Linear Regression
- Đánh giá: R², MSE, MAE

## 4.1. Chia train/test

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42
)

print(f"Train: {X_train.shape}, Test: {X_test.shape}")

## 4.2. Xây dựng model

In [ ]:
lm = LinearRegression()
lm.fit(X_train, y_train)

# Dự đoán
yhat_train = lm.predict(X_train)
yhat_test = lm.predict(X_test)

## 4.3. Đánh giá model

### 4.3.1. R² Score

In [ ]:
print('Full R²:  ', lm.score(X_encoded, y))
print('Train R²: ', lm.score(X_train, y_train))
print('Test R²:  ', lm.score(X_test, y_test))

### 4.3.2. MSE và MAE

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

print('Train MSE:', mean_squared_error(y_train, yhat_train))
print('Test MSE: ', mean_squared_error(y_test, yhat_test))
print('Train MAE:', mean_absolute_error(y_train, yhat_train))
print('Test MAE: ', mean_absolute_error(y_test, yhat_test))

## 4.4. Visualize kết quả

### 4.4.1. So sánh Predicted vs Actual

In [ ]:
plt.scatter(yhat_test, y_test, alpha=0.5)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r-', lw=2)
plt.title('Predicted vs Actual (Test set)')
plt.show()

### 4.4.2. So sánh phân phối

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
ax1 = sns.distplot(y_train, hist=False, color="r", label="Actual Train")
sns.distplot(yhat_train, hist=False, color="b", label="Predicted Train", ax=ax1)
plt.legend()
plt.title('Train Set')

plt.subplot(1,2,2)
ax2 = sns.distplot(y_test, hist=False, color="r", label="Actual Test")
sns.distplot(yhat_test, hist=False, color="b", label="Predicted Test", ax=ax2)
plt.legend()
plt.title('Test Set')

plt.tight_layout()
plt.show()

---
# PHẦN 5: CẢI TIẾN VỚI SELECTKBEST (1 điểm)
**Yêu cầu:**
- Sử dụng SelectKBest chọn k features tốt nhất
- So sánh với model ban đầu

## 5.1. Áp dụng SelectKBest

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

# Chọn k=5 features tốt nhất
selector = SelectKBest(score_func=f_regression, k=5)
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

print(f"Shape ban đầu: {X_train.shape}")
print(f"Shape sau SelectKBest: {X_train_selected.shape}")

## 5.2. Xem feature scores

In [ ]:
# Bảng điểm số features
feature_scores = pd.DataFrame({
    'Feature': X_encoded.columns,
    'Score': selector.scores_
}).sort_values(by='Score', ascending=False)

print("Top 10 features:")
print(feature_scores.head(10))

In [ ]:
# Features được chọn
selected_mask = selector.get_support()
selected_names = X_encoded.columns[selected_mask].tolist()

print(f"\n{selector.k} features được chọn:")
for i, feat in enumerate(selected_names, 1):
    print(f"{i}. {feat}")

## 5.3. Train model mới

In [ ]:
lm_selected = LinearRegression()
lm_selected.fit(X_train_selected, y_train)

yhat_test_selected = lm_selected.predict(X_test_selected)

## 5.4. So sánh 2 models

In [ ]:
print("=" * 50)
print("SO SÁNH 2 MODELS")
print("=" * 50)
print(f"\n1. Model ban đầu ({X_train.shape[1]} features):")
print(f"   Train R²: {lm.score(X_train, y_train):.4f}")
print(f"   Test R²:  {lm.score(X_test, y_test):.4f}")
print(f"   Test MSE: {mean_squared_error(y_test, yhat_test):.2f}")

print(f"\n2. Model với SelectKBest (k={selector.k}):")
print(f"   Train R²: {lm_selected.score(X_train_selected, y_train):.4f}")
print(f"   Test R²:  {lm_selected.score(X_test_selected, y_test):.4f}")
print(f"   Test MSE: {mean_squared_error(y_test, yhat_test_selected):.2f}")

## 5.5. Thử nghiệm với nhiều giá trị k

In [ ]:
k_values = [3, 5, 7, 10]
results = []

for k in k_values:
    selector_k = SelectKBest(score_func=f_regression, k=k)
    X_train_k = selector_k.fit_transform(X_train, y_train)
    X_test_k = selector_k.transform(X_test)
    
    lm_k = LinearRegression()
    lm_k.fit(X_train_k, y_train)
    
    results.append({
        'k': k,
        'R²_train': lm_k.score(X_train_k, y_train),
        'R²_test': lm_k.score(X_test_k, y_test)
    })

results_df = pd.DataFrame(results)
print("\nKết quả với các k khác nhau:")
print(results_df)

In [ ]:
# Biểu đồ
plt.figure(figsize=(10,5))
plt.plot(results_df['k'], results_df['R²_train'], marker='o', label='Train R²')
plt.plot(results_df['k'], results_df['R²_test'], marker='s', label='Test R²')
plt.xlabel('Số lượng features (k)')
plt.ylabel('R²')
plt.title('R² theo số lượng features')
plt.legend()
plt.grid(True)
plt.show()

---
# KẾT LUẬN

## Tóm tắt kết quả:
1. **Phần 1**: Đã đọc và kiểm tra dữ liệu thành công, xử lý duplicate (nếu có)
2. **Phần 2**: 
   - Phân tích đơn biến: Xem phân phối, thống kê mô tả
   - Phân tích đa biến: Ma trận tương quan, kiểm định Chi-square, ANOVA
   - Xử lý outliers bằng phương pháp IQR
3. **Phần 3**: 
   - Chọn features dựa trên correlation >= 0.3 và p-value <= 0.05
   - OneHotEncoder cho biến định tính
   - StandardScaler cho biến định lượng
4. **Phần 4**: 
   - Xây dựng Linear Regression model
   - Đánh giá bằng R², MSE, MAE
   - Trực quan hóa kết quả
5. **Phần 5**: 
   - Cải tiến model bằng SelectKBest
   - So sánh với model ban đầu
   - Tìm giá trị k tối ưu

## Lưu ý quan trọng:
- Luôn kiểm tra duplicate và outliers trước khi modeling
- Chú ý đến tương quan giữa các biến để tránh multicollinearity
- So sánh R² giữa train và test để phát hiện overfitting
- SelectKBest giúp giảm số chiều và cải thiện model